In [1]:
from IPython.display import clear_output

In [2]:
#%pip install azure-ai-evaluation
clear_output()

In [3]:
import os
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from dotenv import load_dotenv

load_dotenv('../../.env')

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
)

# Evaluating Response Quality with RelevanceEvaluator

In [4]:
from azure.ai.evaluation import RelevanceEvaluator

relevance_evaluator = RelevanceEvaluator(model_config)

# Evaluate a single query-response pair
result = relevance_evaluator(
    query="What is the capital of France?",
    response="The capital of France is Paris.",
)

print(result)

{'relevance': 4.0, 'gpt_relevance': 4.0, 'relevance_reason': 'The response fully and accurately answers the question with all essential details, but does not provide any extra insights.', 'relevance_result': 'pass', 'relevance_threshold': 3}


# Evaluating Response Quality with CoherenceEvaluator

In [5]:
from azure.ai.evaluation import CoherenceEvaluator

coherence_evaluator = CoherenceEvaluator(model_config)

result = coherence_evaluator(
    query="What's the capital of France?", 
    response="Paris."
)
print(result)

{'coherence': 4.0, 'gpt_coherence': 4.0, 'coherence_reason': 'The response is fully coherent for the type of question asked, providing a direct and logically connected answer.', 'coherence_result': 'pass', 'coherence_threshold': 3}


# Evaluating Response Quality with FluencyEvaluator

In [6]:
from azure.ai.evaluation import FluencyEvaluator

fluency_evaluator = FluencyEvaluator(model_config)

result = fluency_evaluator(
    response="Paris."
)
print(result)

{'fluency': 1.0, 'gpt_fluency': 1.0, 'fluency_reason': 'The response is a single word and does not demonstrate any command of language, sentence structure, or coherence.', 'fluency_result': 'fail', 'fluency_threshold': 3}


# Evaluating Response Quality with GroundednessEvaluator

In [7]:
from azure.ai.evaluation import GroundednessEvaluator

groundedness_evaluator = GroundednessEvaluator(model_config)

result = groundedness_evaluator(
    query="Who discovered penicillin?",
    context="Alexander Fleming discovered penicillin in 1928 while working at St. Mary's Hospital in London.",
    response="Alexander Fleming discovered penicillin in 1928.",
)
print(result)

{'groundedness': 5.0, 'gpt_groundedness': 5.0, 'groundedness_reason': 'The response is fully correct and complete, directly addressing the query with all relevant details from the context.', 'groundedness_result': 'pass', 'groundedness_threshold': 3}


# Creating custom evaluators

## Code-based evaluator

### Function-based evaluator

In [8]:
# Custom evaluator function to calculate response length
def response_length_evaluator(response, **kwargs):
    return {"response_length": len(response)}

# Example usage
result = response_length_evaluator(response="Hello, world!")
print(result)

{'response_length': 13}


### Class-based evaluator

In [9]:
# Custom class-based evaluator to check for blocked words
class BlocklistEvaluator:
    def __init__(self, blocklist):
        self.blocklist = blocklist

    def __call__(self, *, response: str, **kwargs):
        contains_blocked_word = any(word in response for word in self.blocklist)
        return {"contains_blocked_word": contains_blocked_word}
    
# Example usage
blocklist_evaluator = BlocklistEvaluator(blocklist=["bad", "evil", "worst"])
result = blocklist_evaluator(response="This is the worst response ever!")
print(result)

{'contains_blocked_word': True}


## Prompt-based evaluators

#### Helpfulness evaluator

In [10]:
from helpfulness import HelpfulnessEvaluator

helpfulness_evaluator = HelpfulnessEvaluator(model_config)

helpfulness_score = helpfulness_evaluator(
    query="What's the meaning of life?", 
    context="Arthur Schopenhauer was the first to explicitly ask the question, in an essay entitled 'Character'.", 
    response="The answer is 42."
)
print(helpfulness_score)

{'helpfulness': 1.0, 'helpfulness_reason': "The response is entirely unhelpful, as it does not address the user's question in a meaningful or informative way and ignores the context provided."}


#### JSON accuracy evaluator

In [11]:
import json
from json_schema import JSONSchemaEvaluator

# Load jsons/example.jsonl file here
example_json_schema = json.load(open('jsons/example_schema.json', 'r'))

# Example JSON object
sample_json_data = json.load(open('jsons/poor_output.json', 'r'))

json_schema_evaluator = JSONSchemaEvaluator(model_config)
json_schema_score = json_schema_evaluator(json_output=sample_json_data, schema=example_json_schema)
print(json_schema_score)

{'json_schema': 0.5, 'json_schema_reason': 'The JSON output is almost fully compliant with the schema, but it is missing the required "companyName" field in the "companyInfo" object. All other fields, types, and structures are correct.'}


# Evaluating a dataset

In [12]:
import pandas as pd
from azure.ai.evaluation import evaluate, RetrievalEvaluator
from pprint import pprint
from model_endpoint import ModelEndpoint
from IPython.display import clear_output

azure_ai_project = {
    "subscription_id": os.environ.get("SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("RG_NAME"),
    "project_name": os.environ.get("PROJECT_NAME"),
}

# Define your evaluators
relevance_evaluator = RelevanceEvaluator(model_config)
retrieval_evaluator = RetrievalEvaluator(model_config)

# Evaluate the dataset
result = evaluate(
    data="evaluation_dataset.jsonl",
    target=ModelEndpoint(model_config),
    evaluators={
        # Performance and quality evaluators (AI-assisted)
        "relevance": relevance_evaluator,
        "coherence": coherence_evaluator,
        "fluency": fluency_evaluator,
        "groundedness": groundedness_evaluator,
        "retrieval": retrieval_evaluator,
        # Custom evaluators (code and prompt based)
        "helpfulness": helpfulness_evaluator,
    },
    evaluator_config={
        "relevance": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "coherence": {
            "column_mapping": {"response": "${target.response}", "query": "${data.query}"}
        },
        "fluency": {
            "column_mapping": {"response": "${target.response}"}
        },
        "groundedness": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "retrieval": {
            "column_mapping": {"context": "${data.context}", "query": "${data.query}"}
        },
        "helpfulness": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
    },
    azure_ai_project=azure_ai_project,
    output_path="./evaluation_results.json",
)
clear_output()

In [13]:
pd.DataFrame(result["rows"])

,outputs.query,outputs.response,inputs.query,inputs.context,inputs.ground_truth,outputs.relevance.relevance,outputs.relevance.gpt_relevance,outputs.relevance.relevance_reason,outputs.relevance.relevance_result,outputs.relevance.relevance_threshold,...,outputs.groundedness.groundedness_result,outputs.groundedness.groundedness_threshold,outputs.retrieval.retrieval,outputs.retrieval.gpt_retrieval,outputs.retrieval.retrieval_reason,outputs.retrieval.retrieval_result,outputs.retrieval.retrieval_threshold,outputs.helpfulness.helpfulness,outputs.helpfulness.helpfulness_reason,line_number
0,"What event started on July 28, 1914?","The event that started on **July 28, 1914** wa...","What event started on July 28, 1914?",It involved multiple countries and lasted unti...,World War I,5,5,"The response is fully accurate, complete, and ...",pass,3,...,pass,3,2,2,The context is partially relevant because it g...,fail,3,5,"The response fully answers the question, accur...",0
1,Who was the first person to walk on the moon?,The first person to walk on the moon was **Nei...,Who was the first person to walk on the moon?,The event occurred during the Apollo 11 missio...,Neil Armstrong,5,5,"The response is accurate, complete, and includ...",pass,3,...,pass,3,2,2,The context is partially relevant because it r...,fail,3,5,"The response is fully accurate, complete, and ...",1
2,What was the significance of the year 1776 in ...,The year **1776** is highly significant in Ame...,What was the significance of the year 1776 in ...,A key document was signed declaring independen...,The Declaration of Independence,5,5,The response is not only complete and accurate...,pass,3,...,pass,3,5,5,"The context is highly relevant, directly answe...",pass,3,5,"The response is fully helpful, providing a com...",2
3,"Which wall fell in 1989, symbolizing the end o...","The **Berlin Wall** fell in 1989, symbolizing ...","Which wall fell in 1989, symbolizing the end o...",It divided a German city into East and West.,The Berlin Wall,4,4,The response fully and accurately answers the ...,pass,3,...,pass,3,2,2,The context is partially relevant as it hints ...,fail,3,5,"The response is entirely helpful, directly ans...",3
4,What ancient city was buried by the eruption o...,The ancient city buried by the eruption of Mou...,What ancient city was buried by the eruption o...,The city's ruins were rediscovered in the 18th...,Pompeii,4,4,The response is accurate and fully answers the...,pass,3,...,pass,3,1,1,The context does not provide relevant informat...,fail,3,5,The response fully and accurately answers the ...,4
5,Who was the British Prime Minister during Worl...,The British Prime Minister during most of Worl...,Who was the British Prime Minister during Worl...,"He is famous for his leadership and speeches, ...",Winston Churchill,5,5,"The response is accurate, complete, and includ...",pass,3,...,pass,3,1,1,The context does not provide the name of the B...,fail,3,5,"The response is entirely helpful, accurately n...",5
6,What was the name of the ship that sank on its...,The name of the ship that sank on its maiden v...,What was the name of the ship that sank on its...,It was deemed 'unsinkable' before it hit an ic...,RMS Titanic,4,4,The response fully and accurately answers the ...,pass,3,...,pass,3,2,2,The context is somewhat relevant but does not ...,fail,3,5,"The response is entirely helpful, directly ans...",6
7,Which empire was ruled by Genghis Khan?,Genghis Khan ruled the **Mongol Empire**.,Which empire was ruled by Genghis Khan?,This empire became the largest contiguous land...,The Mongol Empire,4,4,The response fully and accurately answers the ...,pass,3,...,pass,3,5,5,"The context is highly relevant to the query, d...",pass,3,5,The response fully and accurately answers the ...,7
8,What was the primary cause of the American Civ...,The primary cause of the American Civil War wa...,What was the primary cause of the American Civ...,The conflict between the Northern and Southern..

In [14]:
!pip install --upgrade azure-ai-evaluation


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from azure.ai.evaluation import DocumentRetrievalEvaluator

# these query_relevance_label are given by your human- or LLM-judges.
retrieval_ground_truth = [
    {
        "document_id": "1",
        "query_relevance_label": 4
    },
    {
        "document_id": "2",
        "query_relevance_label": 2
    },
    {
        "document_id": "3",
        "query_relevance_label": 3
    },
    {
        "document_id": "4",
        "query_relevance_label": 1
    },
    {
        "document_id": "5",
        "query_relevance_label": 0
    },
]
# the min and max of the label scores are inputs to document retrieval evaluator
ground_truth_label_min = 0
ground_truth_label_max = 4

# these relevance scores come from your search retrieval system
retrieved_documents = [
    {
        "document_id": "2",
        "relevance_score": 45.1
    },
    {
        "document_id": "6",
        "relevance_score": 35.8
    },
    {
        "document_id": "3",
        "relevance_score": 29.2
    },
    {
        "document_id": "5",
        "relevance_score": 25.4
    },
    {
        "document_id": "7",
        "relevance_score": 18.8
    },
]

document_retrieval_evaluator = DocumentRetrievalEvaluator(
    ground_truth_label_min=ground_truth_label_min, 
    ground_truth_label_max=ground_truth_label_max,
    ndcg_threshold = 0.5,
    xdcg_threshold = 50.0,
    fidelity_threshold = 0.5,
    top1_relevance_threshold = 50.0,
    top3_max_relevance_threshold = 50.0,
    total_retrieved_documents_threshold = 50,
    total_ground_truth_documents_threshold = 50
)
document_retrieval_evaluator(retrieval_ground_truth=retrieval_ground_truth, retrieved_documents=retrieved_documents)

{'ndcg@3': 0.31075932533963707,
 'xdcg@3': 39.285714285714285,
 'fidelity': 0.39285714285714285,
 'top1_relevance': 2,
 'top3_max_relevance': 3,
 'holes': 2,
 'holes_ratio': 0.4,
 'total_retrieved_documents': 5,
 'total_ground_truth_documents': 5,
 'ndcg@3_result': 'fail',
 'ndcg@3_threshold': 0.5,
 'ndcg@3_higher_is_better': True,
 'xdcg@3_result': 'fail',
 'xdcg@3_threshold': 50.0,
 'xdcg@3_higher_is_better': True,
 'fidelity_result': 'fail',
 'fidelity_threshold': 0.5,
 'fidelity_higher_is_better': True,
 'top1_relevance_result': 'fail',
 'top1_relevance_threshold': 50.0,
 'top1_relevance_higher_is_better': True,
 'top3_max_relevance_result': 'fail',
 'top3_max_relevance_threshold': 50.0,
 'top3_max_relevance_higher_is_better': True,
 'holes_result': 'fail',
 'holes_threshold': 0,
 'holes_higher_is_better': False,
 'holes_ratio_result': 'fail',
 'holes_ratio_threshold': 0,
 'holes_ratio_higher_is_better': False,
 'total_retrieved_documents_result': 'fail',
 'total_retrieved_document